<a href="https://colab.research.google.com/github/strange221/python_practice/blob/main/Data_Acquisition_CaseStudy_Harisankar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import sqlite3


#Step 1: Load SpaceX Launch Data from API

In [3]:
launch_df = pd.read_json("https://api.spacexdata.com/v4/launches")[['name','date_utc','success','details','rocket']]
launch_df['date_utc']=pd.to_datetime(launch_df['date_utc'])
launch_df['year'] = launch_df['date_utc'].dt.year
launch_df

,name,date_utc,success,details,rocket,year
0,FalconSat,2006-03-24 22:30:00+00:00,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006
1,DemoSat,2007-03-21 01:10:00+00:00,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007
2,Trailblazer,2008-08-03 03:34:00+00:00,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008
3,RatSat,2008-09-28 23:15:00+00:00,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008
4,RazakSat,2009-07-13 03:35:00+00:00,1.0,None,5e9d0d95eda69955f709d1eb,2009
...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022
201,TTL-1,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69974db09d1ed,2022


#Step 2: Load Rocket Metadata

In [4]:
rocket_df = pd.read_json("https://api.spacexdata.com/v4/rockets")[['id','name','type','active','stages']]
rocket_df

,id,name,type,active,stages
0,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
1,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,True,2
3,5e9d0d96eda699382d09d1ee,Starship,rocket,False,2


#Step 3: Merge Launch and Rocket Data

In [5]:
merged_df = launch_df.rename(columns={'name':'Launch_name'}).merge(rocket_df.rename(columns={'name':'Rocket_name'}), left_on='rocket', right_on='id', how='left')
merged_df = merged_df.drop(columns=['id'])
merged_df


,Launch_name,date_utc,success,details,rocket,year,Rocket_name,type,active,stages
0,FalconSat,2006-03-24 22:30:00+00:00,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,Falcon 1,rocket,False,2
1,DemoSat,2007-03-21 01:10:00+00:00,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,Falcon 1,rocket,False,2
2,Trailblazer,2008-08-03 03:34:00+00:00,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,Falcon 1,rocket,False,2
3,RatSat,2008-09-28 23:15:00+00:00,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,Falcon 1,rocket,False,2
4,RazakSat,2009-07-13 03:35:00+00:00,1.0,None,5e9d0d95eda69955f709d1eb,2009,Falcon 1,rocket,False,2
...,...,...,...,...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,Falcon 9,rocket,True,2
201,TTL-1,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,Falcon 9,rocket,True,2
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,Falcon 9,rocket,True,2
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69974db09d1ed,2022,Falcon Heavy,rocket,True,2


#Step 4: Add Simulated Country Information

In [6]:
import random
countries=['USA', 'Russia', 'India', 'China', 'France']
merged_df['Country'] = np.random.choice(countries, len(merged_df))
merged_df

,Launch_name,date_utc,success,details,rocket,year,Rocket_name,type,active,stages,Country
0,FalconSat,2006-03-24 22:30:00+00:00,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,Falcon 1,rocket,False,2,France
1,DemoSat,2007-03-21 01:10:00+00:00,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,Falcon 1,rocket,False,2,Russia
2,Trailblazer,2008-08-03 03:34:00+00:00,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,Falcon 1,rocket,False,2,India
3,RatSat,2008-09-28 23:15:00+00:00,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,Falcon 1,rocket,False,2,Russia
4,RazakSat,2009-07-13 03:35:00+00:00,1.0,None,5e9d0d95eda69955f709d1eb,2009,Falcon 1,rocket,False,2,USA
...,...,...,...,...,...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,Falcon 9,rocket,True,2,USA
201,TTL-1,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,Falcon 9,rocket,True,2,India
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,Falcon 9,rocket,True,2,China
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69974db09d1ed,2022,Falcon Heavy,rocket,True,2,China


#Step 5: Store Merged Data in SQLite3

In [7]:
conn=sqlite3.connect('spacex.db')
merged_df.to_sql("launches", conn, if_exists="replace", index=False)
conn.commit()

#Step 6: Run SQL Queries on the Data to analyze

In [8]:
query1="""SELECT Country, COUNT(*) AS launch_count
FROM launches
GROUP BY Country
ORDER BY launch_count DESC"""
pd.read_sql(query1,conn)

,Country,launch_count
0,USA,46
1,India,43
2,China,42
3,Russia,38
4,France,36


In [9]:
query2="""SELECT year, COUNT(*) AS launch_count
FROM launches
GROUP BY year
ORDER BY launch_count DESC
LIMIT 1"""
pd.read_sql(query2,conn)

,year,launch_count
0,2022,62


In [10]:
query3="""SELECT Launch_name, COUNT(*) AS launch_count
FROM launches
GROUP BY Launch_name
ORDER BY launch_count DESC
LIMIT 5"""
pd.read_sql(query3,conn)

,Launch_name,launch_count
0,ispace Mission 1 & Rashid,1
1,ZUMA,1
2,WorldView Legion 1 & 2,1
3,Viasat-3 & Arcturus,1
4,USSF-44,1


In [11]:
query4="""SELECT Rocket_name, COUNT(*) AS launch_count
FROM launches
GROUP BY Rocket_name
ORDER BY launch_count DESC
LIMIT 1"""
pd.read_sql(query4,conn)

,Rocket_name,launch_count
0,Falcon 9,195
